This notebook focuses on generating spatial transcriptomics plots using the R library SRTsim. I begin by uploading count and coordinate CSV files, which represent gene expression and spatial coordinates, respectively. Using these inputs, I visualize the mean-variance relationship, an important step in understanding variability across spatially resolved transcriptomic data.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import os
import gzip
import numpy as np
import scanpy as sc
import squidpy as sq
import cupy as cp
import cupyx
from cupyx.scipy.sparse import csr_matrix 
import os
import time
import rapids_singlecell as rsc
import numpy as np
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator
import cupy

rmm.reinitialize(
    managed_memory=False,  # Allows oversubscription
    pool_allocator=False,  # default is False
    devices=0,  # GPU device IDs to register. By default registers only GPU 0.
)
cp.cuda.set_allocator(rmm_cupy_allocator)
import zarr
import pickle
from collections import OrderedDict
from scipy.sparse import csr_matrix
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
from scipy.sparse import csr_matrix
import scipy
from anndata import AnnData
from collections import OrderedDict
from rsc_functions.utility.applyqc import applyqc
from rsc_functions.reports.plot import plot_spatial,plot_spatial_data, plot_dist
from rsc_functions.utility.rank_genes_groups import return_markers,rank_genes_groups
from rsc_functions.reports.plot import plot_expression

In [ ]:
counts = pd.read_csv('Rsim_count.csv')

In [ ]:
adata = AnnData(counts.iloc[:,1:].to_numpy(), obsm={"spatial": pd.read_csv('Rsim_centroids.csv').loc[:,['x','y']].to_numpy()})

In [ ]:
mean = np.mean(adata.X,axis=1)
var = np.var(adata.X,axis=1)
covar = var + var/mean
df_var = pd.DataFrame({'mean':mean,'variance':var,'covariance':covar})

In [ ]:
ax = sns.scatterplot(
    data=df_var, x='mean', y="variance", s=5
)

In [ ]:
from skmisc.loess import loess

In [ ]:
estimat_var = cp.zeros(adata.shape[0], dtype=np.float64)
not_const = var > 0

In [ ]:
model = loess(mean, var, span=0.3, degree=2)
model.fit()
estimat_var[not_const] = model.outputs.fitted_values

In [ ]:
reg_std = cp.sqrt(10**estimat_var)